In [17]:
import numpy as np

def pre_sampling(params):
    params[0] = np.random.normal(0, 1, 1)
    return params

def post_sampling(params, m):
    params[1] = np.random.normal(0, 1, m)
    return params

def calc(params, scr):
    return scr * (params[0] + params[1])

def evppi_calc(result, level, m):
    params = np.zeros((2, m))
    params = pre_sampling(params)
    params = post_sampling(params, m)
    
    if level == 0:
        sum_of_max = 0
        sum_of_no_scr = 0
        sum_of_scr = 0
        
        for i in range(m):
            no_scr = calc(params[:, i], scr = 0)
            scr = calc(params[:, i], scr = 1)
            
            sum_of_max += max(no_scr, scr)
            sum_of_no_scr += no_scr
            sum_of_scr += scr
            
        sum_of_max /= m
        max_of_sum = max(sum_of_no_scr, sum_of_scr) / m
        p = sum_of_max - max_of_sum
        
        result[level][0] += p
        result[level][1] += p * p
        result[level][2] += p
        result[level][3] += p * p
    
    else:
        sum_of_max = 0
        sum_of_no_scr_first = 0
        sum_of_scr_first = 0
        sum_of_no_scr_second = 0
        sum_of_scr_second = 0
        
        for i in range(int(m / 2)):
            no_scr = calc(params[:, i], scr = 0)
            scr = calc(params[:, i], scr = 1)
            
            sum_of_max += max(no_scr, scr)
            sum_of_no_scr_first += no_scr
            sum_of_scr_first += scr
            
        for i in range(int(m / 2), m):
            no_scr = calc(params[:, i], scr = 0)
            scr = calc(params[:, i], scr = 1)
            
            sum_of_max += max(no_scr, scr)
            sum_of_no_scr_second += no_scr
            sum_of_scr_second += scr
            
        sum_of_max /= m
        max_of_sum = max(sum_of_no_scr_first + sum_of_no_scr_second, sum_of_scr_first + sum_of_scr_second) / m
        p = sum_of_max - max_of_sum
        result[level][0] += p
        result[level][1] += p * p
        
        max_of_sum_first = max(sum_of_no_scr_first, sum_of_scr_first) / int(m / 2)
        max_of_sum_second = max(sum_of_no_scr_second, sum_of_scr_second) / (m - int(m / 2))
        z = (max_of_sum_first + max_of_sum_second) / 2 - max_of_sum
        result[level][2] += z
        result[level][3] += z * z
        
    return result
    
    
def mlmc_calc(level, m, n_samples):
    result = np.zeros((11, 4))
    for _ in range(n_samples[level]):
        result = evppi_calc(result, level, m)
        
    return result
        

def mlmc_test():
    print('l aveZ aveP varZ varP')
    print('-----------------------------')
    
    m = 1
    n_samples = [2000 for _ in range(11)]
    for level in range(11):
        result = mlmc_calc(level, m, n_samples)
        m *= 2
        
        print(
            str(level) + ' ' + \
            str(result[level][2] / n_samples[level]) + ' ' + \
            str(result[level][0] / n_samples[level]) + ' ' + \
            str(result[level][1] / n_samples[level]) + ' ' + \
            str(result[level][3] / n_samples[level])
        )
        
mlmc_test()

l aveZ aveP varZ varP
-----------------------------
0 0.0 0.0 0.0 0.0
1 0.07295988856053916 0.07295988856053916 0.026974460962270574 0.026974460962270574
2 0.04542107311400443 0.11759144876713028 0.035013970630913896 0.011922926473186378
3 0.023371950560405585 0.14203596437698307 0.038760558284768515 0.003982020048168715
4 0.011439730290584012 0.1516797075463578 0.0395244969429054 0.0014117015637732078
5 0.005735518637940189 0.157785052750327 0.03997584320541894 0.0005060157127879674
6 0.0027182868765236105 0.16335261060773884 0.04052203113999588 0.00018339194182617175
7 0.00180888490048654 0.16297431751590166 0.040507461089299944 9.544092755995894e-05
8 0.0008504155893073025 0.16954874426331254 0.042868956869350515 2.6109793917355733e-05
9 0.0003520276433432014 0.1638858818453796 0.04047552454311535 7.880973940346228e-06
10 0.00020941858095784998 0.16732472432256923 0.0414987644506673 3.264311451590098e-06
